In [1]:
import keras.backend
import keras
from keras.applications.vgg16 import VGG16
from keras.layers import Conv2D,Input
import keras.backend as K
from keras.applications.resnet50 import ResNet50

Using TensorFlow backend.


In [ ]:
class Mask_loc(keras.layers.Layer):
    def __init__(self,maximum_region,W,H):
        self.W = W
        self.H = H
        self.half_H = H//2
        self.half_W = W//2
        self.maximum_region = maximum_region
        
    def call(self,x):
        #image (N,W,H,RGB)
        N = x.shape[0]
        image = x[1]
        #center coordinate (N,self.maxi_region,2)
        output_region = x[2]
        regional_prop = K.Variable((N,self.maximum_region,4))
        for i in range(N) :
            for j in range(self.maximum_region):
                regional_prop[i,j,0] = output_region[i,j,0]-self.half_W #x left
                regional_prop[i,j,1] = output_region[i,j,1]-self.half_H #y bottom
                regional_prop[i,j,2] = output_region[i,j,0]+self.half_W #x right
                regional_prop[i,j,3] = output_region[i,j,1]+self.half_H #y top
        #(N,K,4)
        Mask = K.Variable((N,self.maximum_region,self.W,self.H,3))
        for n in range(N):
            for k in range(self.maximum_region):
                for i in range(self.W):
                    for j in range(self.H):
                        Mask[n,k,i,j,:] = (K.sigmoid(i-regional_prop[n,k,0])-K.sigmoid(i-regional_prop[n,k,2]))*(K.sigmoid(j-regional_prop[n,k,3])-K.sigmoid(j-regional_prop[n,k,1]))
        #(N,K,W,H,3)
        image = K.expand_dims(x,1)
        
        z = K.multiply(image,Mask)
        return z
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0][0], input_shape[0][1], input_shape[0][2],input_shape[0][3])
    
    def build(self, input_shape):
        super(Mask_loc, self).build(input_shape)

In [2]:
from keras.models.Models import build_encoder

ModuleNotFoundError: No module named 'keras.models.Models'; 'keras.models' is not a package

In [41]:
import keras.backend
import keras
from keras.applications.vgg16 import VGG16
from keras.layers import Conv2D,Input
import keras.backend as K
from keras.applications.resnet50 import ResNet50
from keras.engine.topology import InputSpec,Layer
class Mask_loc(Layer):
    def __init__(self,maximum_region,W,H,**kw):
        super(MyLayer, self).__init__(**kw) 
        self.W = W
        self.H = H
        self.half_H = H//2
        self.half_W = W//2
        self.maximum_region = maximum_region
        self.name = 'Mask_loc'
        
    def call(self,x):
        #image (N,W,H,RGB)
        N = x.shape[0]
        image = x[1]
        #center coordinate (N,self.maxi_region,2)
        output_region = x[2]
        regional_prop = K.Variable((N,self.maximum_region,4))
        for i in range(N) :
            for j in range(self.maximum_region):
                regional_prop[i,j,0] = output_region[i,j,0]-self.half_W #x left
                regional_prop[i,j,1] = output_region[i,j,1]-self.half_H #y bottom
                regional_prop[i,j,2] = output_region[i,j,0]+self.half_W #x right
                regional_prop[i,j,3] = output_region[i,j,1]+self.half_H #y top
        #(N,K,4)
        Mask = K.Variable((N,self.maximum_region,self.W,self.H,3))
        for n in range(N):
            for k in range(self.maximum_region):
                for i in range(self.W):
                    for j in range(self.H):
                        Mask[n,k,i,j,:] = (K.sigmoid(i-regional_prop[n,k,0])-K.sigmoid(i-regional_prop[n,k,2]))*(K.sigmoid(j-regional_prop[n,k,3])-K.sigmoid(j-regional_prop[n,k,1]))
        #(N,K,W,H,3)
        image = K.expand_dims(x,1)
        
        z = K.multiply(image,Mask)
        return z
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0][0], input_shape[0][1], input_shape[0][2],input_shape[0][3])
    
    def built(self, input_shape):
        super(Mask_loc, self).build(input_shape)
                

class DAE(keras.models.Model):

    def __init__(self,input_shape,max_region = 4):
        self.shape = input_shape
        #(3,256,256)
        self.max_region = 4
        self.w = input_shape[1]
        self.h = input_shape[2]
        image = Input(shape = self.shape,name = 'input')
        
        options = {
            "activation": "relu",
            "kernel_size": (3, 3),
            "padding": "same"
        }
        
        self.encoder = self.build_encoder()
        output_features = self.encoder(image)
            
        conv_3x3 = Conv2D(
            filters=512,
            name="3x3",
            **options
        )(output_features)
        
        
        output_region = Conv2D(
            filters=max_region * 2,
            kernel_size=(1, 1),
            activation="linear",
            kernel_initializer="zero",
            name="region_proposal"
        )(conv_3x3)
        
        # To do location_function
        output_proposal_regions = Mask_loc(
            maximum_region = self.max_region,
            H = self.h,
            W = self.w
        #   minimum_size=minimum_size
        )([image,output_region])
        #把locate 到的feature map stack在一起
        output_proposal_regions = K.reshape(output_proposal_regions,(-1,self.w,self.h,3))
        #TO DO  encoder
        outputs =  self.encoder()(output_proposal_regions)

        super(DAE,self).__init__(image,outputs)
        
    def build_encoder(self):
        model = ResNet50(weights= 'imagenet',include_top = False)
        
        return model
    
    def compile(self, optimizer, **kwargs):
        return super(DAE, self).compile(optimizer, None)
    
    def predict(self, x, batch_size=None, verbose=0, steps=None):
        return super(DAE, self).predict(x, batch_size, verbose, steps)

In [42]:
dae = DAE(input_shape=(256,256,3),max_region=4)

<class 'keras.engine.training.Model'>


AttributeError: 'Mask_loc' object has no attribute 'input_spec'

In [36]:
from keras.engine.topology import Layer

In [37]:
from keras import backend as K
from keras.engine.topology import Layer
import numpy as np

class MyLayer(Layer):

    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(MyLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.kernel = self.add_weight(name='kernel', 
                                      shape=(input_shape[1], self.output_dim),
                                      initializer='uniform',
                                      trainable=True)
        super(MyLayer, self).build(input_shape)  # Be sure to call this somewhere!

    def call(self, x):
        return K.dot(x, self.kernel)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

In [38]:
layer = MyLayer((3,3,3))

In [43]:
class MyLayer(Layer):    
    def __init__(self, output_dim, **kw):    
        self.output_dim = output_dim   
        self.SCALER = None  
        super(MyLayer, self).__init__(**kw)    
    def build(self, input_shape):    
        input_dim = input_shape[1]   
        self.SCALER=self.add_weight(shape=(input_dim,), initializer='uniform', trainable=True)#注意shape写法  
        super(MyLayer, self).build(input_shape)    
    def call(self, x):    
        x *= self.SCALER  
        return x    
    def compute_output_shape(self, input_shape):  
        return input_shape  
  
input_data=np.random.randint(0, 10, (3000, 2))  
test_data=np.random.randint(0, 10, (1, 2))    #test_data=[4, 1]  
labels = 0.1*input_data  
model = Sequential()  
model.add(MyLayer(2, input_shape=(2,)))    
model.compile(optimizer='rmsprop', loss='mse')  
model.fit(input_data, labels, verbose=1)  
res=model.predict(test_data)                   #res=[0.40302727, 0.09999914]  
model.get_weights()                            #weights=[0.10075682, 0.09999914]

NameError: name 'Sequential' is not defined

In [44]:
from keras.models import Sequential  
import numpy as np  
from keras.engine.topology import Layer  
  
np.random.seed(19931221)  
  
class MyLayer(Layer):    
    def __init__(self, output_dim, **kw):    
        self.output_dim = output_dim   
        self.SCALER = None  
        super(MyLayer, self).__init__(**kw)    
    def build(self, input_shape):    
        input_dim = input_shape[1]   
        self.SCALER=self.add_weight(shape=(input_dim,), initializer='uniform', trainable=True)#注意shape写法  
        super(MyLayer, self).build(input_shape)    
    def call(self, x):    
        x *= self.SCALER  
        return x    
    def compute_output_shape(self, input_shape):  
        return input_shape  
  
input_data=np.random.randint(0, 10, (3000, 2))  
test_data=np.random.randint(0, 10, (1, 2))    #test_data=[4, 1]  
labels = 0.1*input_data  
model = Sequential()  
model.add(MyLayer(2, input_shape=(2,)))    
model.compile(optimizer='rmsprop', loss='mse')  
model.fit(input_data, labels, verbose=1)  
res=model.predict(test_data)                   #res=[0.40302727, 0.09999914]  
model.get_weights()                            #weights=[0.10075682, 0.09999914]

TypeError: add_weight() missing 1 required positional argument: 'name'

In [48]:
from glob import glob

In [63]:
path = glob('/Users/Mike/tensorflow/horse2zebra/trainA/*')

In [64]:
path

['/Users/Mike/tensorflow/horse2zebra/trainA/n02381460_1001.jpg',
 '/Users/Mike/tensorflow/horse2zebra/trainA/n02381460_1002.jpg',
 '/Users/Mike/tensorflow/horse2zebra/trainA/n02381460_1003.jpg',
 '/Users/Mike/tensorflow/horse2zebra/trainA/n02381460_1006.jpg',
 '/Users/Mike/tensorflow/horse2zebra/trainA/n02381460_1008.jpg',
 '/Users/Mike/tensorflow/horse2zebra/trainA/n02381460_1009.jpg',
 '/Users/Mike/tensorflow/horse2zebra/trainA/n02381460_1011.jpg',
 '/Users/Mike/tensorflow/horse2zebra/trainA/n02381460_1014.jpg',
 '/Users/Mike/tensorflow/horse2zebra/trainA/n02381460_1019.jpg',
 '/Users/Mike/tensorflow/horse2zebra/trainA/n02381460_102.jpg',
 '/Users/Mike/tensorflow/horse2zebra/trainA/n02381460_1023.jpg',
 '/Users/Mike/tensorflow/horse2zebra/trainA/n02381460_1025.jpg',
 '/Users/Mike/tensorflow/horse2zebra/trainA/n02381460_1027.jpg',
 '/Users/Mike/tensorflow/horse2zebra/trainA/n02381460_1028.jpg',
 '/Users/Mike/tensorflow/horse2zebra/trainA/n02381460_1034.jpg',
 '/Users/Mike/tensorflow/h

In [65]:
import scipy

In [66]:
import numpy as np

In [ ]:
np.fliplr()